In [1]:
import csv
import numpy
import os
import yaml
import time

In [2]:
from nupic.algorithms.sdr_classifier_factory import SDRClassifierFactory
from nupic.algorithms.spatial_pooler import SpatialPooler
from nupic.algorithms.temporal_memory import TemporalMemory
from nupic.encoders.random_distributed_scalar import RandomDistributedScalarEncoder

In [3]:
_NUM_RECORDS = 14400

_WORK_DIR = os.getcwd()
_INPUT_FILE_PATH = os.path.join(_WORK_DIR, "dataset", "eyeState14400.csv")
_PARAMS_PATH = os.path.join(_WORK_DIR, "params", "modelEyeState.yaml")

print _INPUT_FILE_PATH
print _PARAMS_PATH

E:\vo\kuliahS1\ta\eyeStateHTM\dataset\eyeState14400.csv
E:\vo\kuliahS1\ta\eyeStateHTM\params\modelEyeState.yaml


In [66]:
with open(_PARAMS_PATH, "r") as f:
    modelParams = yaml.safe_load(f)["modelParams"]
    enParams = modelParams["sensorParams"]["encoders"]
    spParams = modelParams["spParams"]
    tmParams = modelParams["tmParams"]

In [67]:
encoderAF3 = RandomDistributedScalarEncoder(enParams["sensAF3"]["resolution"])
encoderF7 = RandomDistributedScalarEncoder(enParams["sensF7"]["resolution"])
encoderF3 = RandomDistributedScalarEncoder(enParams["sensF3"]["resolution"])
encoderFC5 = RandomDistributedScalarEncoder(enParams["sensFC5"]["resolution"])
encoderT7 = RandomDistributedScalarEncoder(enParams["sensT7"]["resolution"])
encoderP7 = RandomDistributedScalarEncoder(enParams["sensP7"]["resolution"])
encoderO1 = RandomDistributedScalarEncoder(enParams["sensO1"]["resolution"])
encoderO2 = RandomDistributedScalarEncoder(enParams["sensO2"]["resolution"])
encoderP8 = RandomDistributedScalarEncoder(enParams["sensP8"]["resolution"])
encoderT8 = RandomDistributedScalarEncoder(enParams["sensT8"]["resolution"])
encoderFC6 = RandomDistributedScalarEncoder(enParams["sensFC6"]["resolution"])
encoderF4 = RandomDistributedScalarEncoder(enParams["sensF4"]["resolution"])
encoderF8 = RandomDistributedScalarEncoder(enParams["sensF8"]["resolution"])
encoderAF4 = RandomDistributedScalarEncoder(enParams["sensAF4"]["resolution"])
encoderEyeDetection = RandomDistributedScalarEncoder(enParams["sensEyeDetection"]["resolution"])

encodingWidth = (encoderAF3.getWidth() + encoderF7.getWidth() + encoderF3.getWidth()
                 + encoderFC5.getWidth() + encoderT7.getWidth() + encoderP7.getWidth()
                 + encoderO1.getWidth() + encoderO2.getWidth() + encoderP8.getWidth()
                 + encoderT8.getWidth() + encoderFC6.getWidth() + encoderF4.getWidth()
                 + encoderF8.getWidth() + encoderAF4.getWidth() + encoderEyeDetection.getWidth())

print encodingWidth

6000


In [68]:
start = time.time()

sp = SpatialPooler(
  inputDimensions=(encodingWidth,),
  columnDimensions=(spParams["columnCount"],),
  potentialPct=spParams["potentialPct"],
  potentialRadius=encodingWidth,
  globalInhibition=spParams["globalInhibition"],
  localAreaDensity=spParams["localAreaDensity"],
  numActiveColumnsPerInhArea=spParams["numActiveColumnsPerInhArea"],
  synPermInactiveDec=spParams["synPermInactiveDec"],
  synPermActiveInc=spParams["synPermActiveInc"],
  synPermConnected=spParams["synPermConnected"],
  boostStrength=spParams["boostStrength"],
  seed=spParams["seed"],
  wrapAround=False
)

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

00:00:54.44


In [69]:
tm = TemporalMemory(
  columnDimensions=(tmParams["columnCount"],),
  cellsPerColumn=tmParams["cellsPerColumn"],
  activationThreshold=tmParams["activationThreshold"],
  initialPermanence=tmParams["initialPerm"],
  connectedPermanence=spParams["synPermConnected"],
  minThreshold=tmParams["minThreshold"],
  maxNewSynapseCount=tmParams["newSynapseCount"],
  permanenceIncrement=tmParams["permanenceInc"],
  permanenceDecrement=tmParams["permanenceDec"],
  predictedSegmentDecrement=0.0,
  maxSegmentsPerCell=tmParams["maxSegmentsPerCell"],
  maxSynapsesPerSegment=tmParams["maxSynapsesPerSegment"],
  seed=tmParams["seed"]
)

In [70]:
classifier = SDRClassifierFactory.create()
wrongDetect = 0

In [71]:
with open(_INPUT_FILE_PATH, "r") as fin:
  reader = csv.reader(fin)
  headers = reader.next()
  reader.next()
  reader.next()

  for count, record in enumerate(reader):

    if count >= _NUM_RECORDS: break
    
    sensAF3 = float(record[0])
    sensF7 = float(record[1])
    sensF3 = float(record[2])
    sensFC5 = float(record[3])
    sensT7 = float(record[4])
    sensP7 = float(record[5])
    sensO1 = float(record[6])
    sensO2 = float(record[7])
    sensP8 = float(record[8])
    sensT8 = float(record[9])
    sensFC6 = float(record[10])
    sensF4 = float(record[11])
    sensF8 = float(record[12])
    sensAF4 = float(record[13])
    sensEyeDetection = float(record[14])
    
    sensAF3_Bits = numpy.zeros(encoderAF3.getWidth())
    sensF7_Bits = numpy.zeros(encoderF7.getWidth()) 
    sensF3_Bits = numpy.zeros(encoderF3.getWidth()) 
    sensFC5_Bits = numpy.zeros(encoderFC5.getWidth())
    sensT7_Bits = numpy.zeros(encoderT7.getWidth()) 
    sensP7_Bits = numpy.zeros(encoderP7.getWidth()) 
    sensO1_Bits = numpy.zeros(encoderO1.getWidth()) 
    sensO2_Bits = numpy.zeros(encoderO2.getWidth()) 
    sensP8_Bits = numpy.zeros(encoderP8.getWidth()) 
    sensT8_Bits = numpy.zeros(encoderT8.getWidth()) 
    sensFC6_Bits = numpy.zeros(encoderFC6.getWidth())
    sensF4_Bits = numpy.zeros(encoderF4.getWidth()) 
    sensF8_Bits = numpy.zeros(encoderF8.getWidth()) 
    sensAF4_Bits = numpy.zeros(encoderAF4.getWidth())
    sensEyeDetection_Bits = numpy.zeros(encoderEyeDetection.getWidth())
    
    encoderAF3.encodeIntoArray(sensAF3, sensAF3_Bits)
    encoderF7.encodeIntoArray(sensF7, sensF7_Bits)
    encoderF3.encodeIntoArray(sensF3, sensF3_Bits)
    encoderFC5.encodeIntoArray(sensFC5, sensFC5_Bits)
    encoderT7.encodeIntoArray(sensT7, sensT7_Bits)
    encoderP7.encodeIntoArray(sensP7, sensP7_Bits)
    encoderO1.encodeIntoArray(sensO1, sensO1_Bits)
    encoderO2.encodeIntoArray(sensO2, sensO2_Bits)
    encoderP8.encodeIntoArray(sensP8, sensP8_Bits)
    encoderT8.encodeIntoArray(sensT8, sensT8_Bits)
    encoderFC6.encodeIntoArray(sensFC6, sensFC6_Bits)
    encoderF4.encodeIntoArray(sensF4, sensF4_Bits)
    encoderF8.encodeIntoArray(sensF8, sensF8_Bits)
    encoderAF4.encodeIntoArray(sensAF4, sensAF4_Bits)
    encoderEyeDetection.encodeIntoArray(sensEyeDetection, sensEyeDetection_Bits)

    encoding = numpy.concatenate(
        [sensAF3_Bits, sensF7_Bits, sensF3_Bits, sensFC5_Bits, sensT7_Bits,
         sensP7_Bits, sensO1_Bits, sensO2_Bits, sensP8_Bits, sensT8_Bits,
         sensFC6_Bits, sensF4_Bits, sensF8_Bits, sensAF4_Bits, sensEyeDetection_Bits]
    )
    
    activeColumns = numpy.zeros(spParams["columnCount"])
    
    # Execute Spatial Pooling algorithm over input space.
    sp.compute(encoding, True, activeColumns)
    activeColumnIndices = numpy.nonzero(activeColumns)[0]

    # Execute Temporal Memory algorithm over active mini-columns.
    tm.compute(activeColumnIndices, learn=True)

    activeCells = tm.getActiveCells()
    
    #print encoding
    
    # Get the bucket info for this input value for classification.
    bucketIdx = encoderEyeDetection.getBucketIndices(sensEyeDetection)[0]
    
    # Run classifier to translate active cells back to scalar value.
    classifierResult = classifier.compute(
      recordNum=count,
      patternNZ=activeCells,
      classification={
        "bucketIdx": bucketIdx,
        "actValue": sensEyeDetection
      },
      learn=True,
      infer=True
    )

    # Print the best prediction for 1 step out.
    oneStepConfidence, oneStep = sorted(
      zip(classifierResult[1], classifierResult["actualValues"]),
      reverse=True
    )[0]
    print("{:5}\t{:4}\t{:4}\t({:4.4}%)".format(count, sensEyeDetection, oneStep, oneStepConfidence * 100))
    #results.append([oneStep, oneStepConfidence * 100, None, None])
    if (sensEyeDetection == oneStep) : wrongDetect += 1
    #print classifierResult



    0	 0.0	 0.0	(100.0%)
    1	 0.0	 0.0	(0.1996%)
    2	 0.0	 0.0	(0.1996%)
    3	 0.0	 0.0	(0.1996%)
    4	 0.0	 0.0	(0.1996%)
    5	 0.0	 0.0	(0.1996%)
    6	 0.0	 0.0	(0.1996%)
    7	 0.0	 0.0	(0.1996%)
    8	 0.0	 0.0	(0.1996%)
    9	 0.0	 0.0	(0.1996%)
   10	 0.0	 0.0	(0.2061%)
   11	 0.0	 0.0	(0.1996%)
   12	 0.0	 0.0	(0.266%)
   13	 0.0	 0.0	(0.2747%)
   14	 0.0	 0.0	(0.2746%)
   15	 0.0	 0.0	(0.8632%)
   16	 0.0	 0.0	(0.7136%)
   17	 0.0	 0.0	(0.9193%)
   18	 0.0	 0.0	(0.836%)
   19	 0.0	 0.0	(0.8099%)
   20	 0.0	 0.0	(0.7142%)
   21	 0.0	 0.0	(0.6919%)
   22	 0.0	 0.0	(1.22%)
   23	 0.0	 0.0	(1.258%)
   24	 0.0	 0.0	(4.089%)
   25	 0.0	 0.0	(5.337%)
   26	 0.0	 0.0	(1.803%)
   27	 0.0	 0.0	(5.293%)
   28	 0.0	 0.0	(4.82%)
   29	 0.0	 0.0	(7.892%)
   30	 0.0	 0.0	(6.128%)
   31	 0.0	 0.0	(4.177%)
   32	 0.0	 0.0	(8.249%)
   33	 0.0	 0.0	(5.294%)
   34	 0.0	 0.0	(5.408%)
   35	 0.0	 0.0	(11.08%)
   36	 0.0	 0.0	(22.76%)
   37	 0.0	 0.0	(2.007%)
   38	 0.0	 0.0	(25.54%)
   39	 0

  328	 1.0	 1.0	(46.72%)
  329	 1.0	 1.0	(1.876%)
  330	 1.0	 1.0	(7.164%)
  331	 1.0	 0.0	(57.6%)
  332	 1.0	 0.0	(0.5562%)
  333	 1.0	 1.0	(0.4524%)
  334	 1.0	 1.0	(50.98%)
  335	 1.0	 1.0	(0.7595%)
  336	 1.0	 1.0	(9.55%)
  337	 1.0	 1.0	(95.09%)
  338	 1.0	 1.0	(5.082%)
  339	 1.0	 0.0	(50.62%)
  340	 1.0	 0.0	(1.23%)
  341	 1.0	 1.0	(5.923%)
  342	 1.0	 1.0	(72.84%)
  343	 1.0	 1.0	(0.6989%)
  344	 1.0	 1.0	(2.42%)
  345	 1.0	 1.0	(76.24%)
  346	 1.0	 1.0	(89.77%)
  347	 1.0	 1.0	(54.97%)
  348	 1.0	 1.0	(0.6955%)
  349	 1.0	 0.0	(20.16%)
  350	 1.0	 1.0	(55.82%)
  351	 1.0	 0.0	(1.916%)
  352	 1.0	 1.0	(38.99%)
  353	 1.0	 1.0	(0.7591%)
  354	 1.0	 1.0	(26.22%)
  355	 1.0	 0.0	(3.398%)
  356	 1.0	 1.0	(26.27%)
  357	 1.0	 1.0	(1.603%)
  358	 1.0	 1.0	(1.882%)
  359	 1.0	 0.0	(2.261%)
  360	 1.0	 0.0	(60.84%)
  361	 1.0	 0.0	(3.818%)
  362	 1.0	 0.0	(1.173%)
  363	 1.0	 0.0	(1.277%)
  364	 1.0	 0.0	(2.922%)
  365	 1.0	 0.0	(0.372%)
  366	 1.0	 1.0	(0.2742%)
  367	 1.0	 0.0	(0.325

  657	 1.0	 1.0	(96.63%)
  658	 1.0	 1.0	(99.88%)
  659	 1.0	 1.0	(98.89%)
  660	 1.0	 1.0	(98.71%)
  661	 1.0	 1.0	(98.95%)
  662	 1.0	 1.0	(99.76%)
  663	 1.0	 1.0	(99.69%)
  664	 1.0	 1.0	(99.48%)
  665	 1.0	 1.0	(99.96%)
  666	 1.0	 1.0	(99.72%)
  667	 1.0	 1.0	(95.18%)
  668	 1.0	 1.0	(95.52%)
  669	 1.0	 1.0	(84.0%)
  670	 1.0	 1.0	(99.54%)
  671	 1.0	 1.0	(99.41%)
  672	 1.0	 1.0	(99.81%)
  673	 1.0	 1.0	(99.91%)
  674	 1.0	 1.0	(99.74%)
  675	 1.0	 1.0	(84.93%)
  676	 1.0	 1.0	(97.27%)
  677	 1.0	 1.0	(99.77%)
  678	 1.0	 1.0	(99.43%)
  679	 1.0	 1.0	(99.67%)
  680	 1.0	 1.0	(98.82%)
  681	 1.0	 1.0	(97.35%)
  682	 1.0	 1.0	(99.9%)
  683	 1.0	 1.0	(98.69%)
  684	 1.0	 1.0	(99.89%)
  685	 1.0	 1.0	(85.61%)
  686	 1.0	 1.0	(99.01%)
  687	 1.0	 1.0	(84.85%)
  688	 1.0	 1.0	(99.37%)
  689	 1.0	 1.0	(85.7%)
  690	 1.0	 1.0	(99.93%)
  691	 1.0	 1.0	(99.9%)
  692	 1.0	 1.0	(99.86%)
  693	 1.0	 1.0	(99.09%)
  694	 1.0	 1.0	(99.78%)
  695	 1.0	 1.0	(99.29%)
  696	 1.0	 1.0	(91.58%)
  69

  985	 0.0	 0.0	(65.19%)
  986	 0.0	 0.0	(99.94%)
  987	 0.0	 0.0	(90.37%)
  988	 0.0	 0.0	(14.02%)
  989	 0.0	 1.0	(2.719%)
  990	 0.0	 0.0	(3.049%)
  991	 0.0	 0.0	(0.8771%)
  992	 0.0	 1.0	(1.298%)
  993	 0.0	 1.0	(10.99%)
  994	 0.0	 1.0	(10.81%)
  995	 0.0	 1.0	(98.92%)
  996	 0.0	 1.0	(92.23%)
  997	 0.0	 0.0	(49.87%)
  998	 0.0	 1.0	(76.43%)
  999	 0.0	 1.0	(25.3%)
 1000	 0.0	 0.0	(65.71%)
 1001	 0.0	 1.0	(1.684%)
 1002	 0.0	 0.0	(82.16%)
 1003	 0.0	 0.0	(14.12%)
 1004	 0.0	 0.0	(5.144%)
 1005	 0.0	 0.0	(63.92%)
 1006	 0.0	 1.0	(9.692%)
 1007	 0.0	 1.0	(91.8%)
 1008	 0.0	 0.0	(88.41%)
 1009	 0.0	 0.0	(61.89%)
 1010	 0.0	 1.0	(3.623%)
 1011	 0.0	 0.0	(21.38%)
 1012	 0.0	 1.0	(85.84%)
 1013	 0.0	 1.0	(97.54%)
 1014	 0.0	 1.0	(75.56%)
 1015	 0.0	 0.0	(69.34%)
 1016	 0.0	 1.0	(14.2%)
 1017	 0.0	 0.0	(89.1%)
 1018	 0.0	 0.0	(55.17%)
 1019	 0.0	 0.0	(89.99%)
 1020	 0.0	 0.0	(4.084%)
 1021	 0.0	 0.0	(83.02%)
 1022	 0.0	 1.0	(2.421%)
 1023	 0.0	 0.0	(83.47%)
 1024	 0.0	 0.0	(4.098%)
 10

 1316	 1.0	 1.0	(1.391%)
 1317	 1.0	 1.0	(1.423%)
 1318	 1.0	 1.0	(1.338%)
 1319	 1.0	 1.0	(2.349%)
 1320	 1.0	 1.0	(1.914%)
 1321	 1.0	 1.0	(34.17%)
 1322	 1.0	 1.0	(99.99%)
 1323	 1.0	 1.0	(99.7%)
 1324	 1.0	 1.0	(81.1%)
 1325	 1.0	 1.0	(99.13%)
 1326	 1.0	 1.0	(1.366%)
 1327	 1.0	 1.0	(98.25%)
 1328	 1.0	 1.0	(1.459%)
 1329	 1.0	 1.0	(1.289%)
 1330	 1.0	 1.0	(2.135%)
 1331	 1.0	 1.0	(1.131%)
 1332	 1.0	 1.0	(2.524%)
 1333	 1.0	 1.0	(2.237%)
 1334	 1.0	 1.0	(3.289%)
 1335	 1.0	 1.0	(99.98%)
 1336	 1.0	 1.0	(99.96%)
 1337	 1.0	 1.0	(30.9%)
 1338	 1.0	 1.0	(99.74%)
 1339	 1.0	 0.0	(77.65%)
 1340	 1.0	 0.0	(93.24%)
 1341	 1.0	 1.0	(6.929%)
 1342	 1.0	 1.0	(99.59%)
 1343	 1.0	 1.0	(4.591%)
 1344	 1.0	 1.0	(99.88%)
 1345	 1.0	 1.0	(1.275%)
 1346	 1.0	 1.0	(1.174%)
 1347	 1.0	 1.0	(87.23%)
 1348	 1.0	 1.0	(5.001%)
 1349	 1.0	 1.0	(98.57%)
 1350	 1.0	 1.0	(2.584%)
 1351	 1.0	 1.0	(17.56%)
 1352	 1.0	 1.0	(99.86%)
 1353	 1.0	 1.0	(89.28%)
 1354	 1.0	 1.0	(99.97%)
 1355	 1.0	 1.0	(17.9%)
 135

...

12848	 0.0	 0.0	(99.8%)
12849	 0.0	 0.0	(99.49%)
12850	 0.0	 0.0	(88.64%)
12851	 0.0	 0.0	(99.84%)
12852	 0.0	 0.0	(99.96%)
12853	 0.0	 0.0	(100.0%)
12854	 0.0	 0.0	(99.96%)
12855	 0.0	 0.0	(96.09%)
12856	 0.0	 0.0	(99.98%)
12857	 0.0	 0.0	(99.99%)
12858	 0.0	 0.0	(97.07%)
12859	 0.0	 0.0	(93.66%)
12860	 0.0	 0.0	(96.59%)
12861	 0.0	 0.0	(98.76%)
12862	 0.0	 0.0	(99.8%)
12863	 0.0	 0.0	(98.64%)
12864	 0.0	 0.0	(99.98%)
12865	 0.0	 0.0	(65.39%)
12866	 0.0	 0.0	(99.85%)
12867	 0.0	 0.0	(99.55%)
12868	 0.0	 0.0	(99.99%)
12869	 0.0	 0.0	(99.95%)
12870	 0.0	 0.0	(99.31%)
12871	 0.0	 0.0	(95.85%)
12872	 0.0	 0.0	(99.43%)
12873	 0.0	 0.0	(99.98%)
12874	 0.0	 0.0	(97.24%)
12875	 0.0	 0.0	(63.84%)
12876	 0.0	 0.0	(79.47%)
12877	 0.0	 0.0	(84.52%)
12878	 0.0	 0.0	(99.87%)
12879	 0.0	 0.0	(70.26%)
12880	 0.0	 0.0	(97.52%)
12881	 0.0	 0.0	(99.84%)
12882	 0.0	 0.0	(99.19%)
12883	 0.0	 0.0	(96.51%)
12884	 0.0	 0.0	(92.47%)
12885	 0.0	 0.0	(99.97%)
12886	 0.0	 0.0	(99.91%)
12887	 0.0	 0.0	(99.62%)
12

13177	 0.0	 0.0	(99.95%)
13178	 0.0	 0.0	(99.96%)
13179	 0.0	 0.0	(99.08%)
13180	 0.0	 0.0	(97.81%)
13181	 0.0	 0.0	(98.49%)
13182	 0.0	 0.0	(99.87%)
13183	 0.0	 0.0	(98.47%)
13184	 0.0	 0.0	(99.11%)
13185	 0.0	 0.0	(97.62%)
13186	 0.0	 0.0	(99.13%)
13187	 0.0	 0.0	(99.88%)
13188	 0.0	 0.0	(99.05%)
13189	 0.0	 0.0	(99.13%)
13190	 0.0	 0.0	(99.84%)
13191	 0.0	 0.0	(99.96%)
13192	 0.0	 0.0	(99.91%)
13193	 0.0	 0.0	(98.32%)
13194	 0.0	 0.0	(99.88%)
13195	 0.0	 0.0	(99.23%)
13196	 0.0	 0.0	(99.64%)
13197	 0.0	 0.0	(99.53%)
13198	 0.0	 0.0	(99.94%)
13199	 0.0	 0.0	(99.75%)
13200	 0.0	 0.0	(95.48%)
13201	 0.0	 0.0	(99.27%)
13202	 0.0	 0.0	(98.7%)
13203	 0.0	 0.0	(97.77%)
13204	 0.0	 0.0	(97.0%)
13205	 0.0	 0.0	(98.6%)
13206	 0.0	 0.0	(99.95%)
13207	 0.0	 0.0	(99.72%)
13208	 0.0	 0.0	(95.63%)
13209	 0.0	 0.0	(98.45%)
13210	 0.0	 0.0	(99.79%)
13211	 0.0	 0.0	(37.51%)
13212	 0.0	 0.0	(99.95%)
13213	 0.0	 0.0	(10.59%)
13214	 0.0	 0.0	(99.71%)
13215	 0.0	 0.0	(99.6%)
13216	 0.0	 0.0	(96.52%)
1321

13506	 0.0	 0.0	(99.89%)
13507	 0.0	 0.0	(99.89%)
13508	 0.0	 0.0	(98.44%)
13509	 0.0	 0.0	(99.78%)
13510	 0.0	 0.0	(98.29%)
13511	 0.0	 0.0	(96.67%)
13512	 0.0	 0.0	(99.87%)
13513	 0.0	 0.0	(99.29%)
13514	 0.0	 0.0	(96.65%)
13515	 0.0	 0.0	(99.09%)
13516	 0.0	 0.0	(99.62%)
13517	 0.0	 0.0	(99.96%)
13518	 0.0	 0.0	(99.99%)
13519	 0.0	 0.0	(99.96%)
13520	 0.0	 0.0	(99.97%)
13521	 0.0	 0.0	(99.99%)
13522	 0.0	 0.0	(100.0%)
13523	 0.0	 0.0	(99.99%)
13524	 0.0	 0.0	(99.86%)
13525	 0.0	 0.0	(99.6%)
13526	 0.0	 0.0	(99.93%)
13527	 0.0	 0.0	(99.99%)
13528	 0.0	 0.0	(99.34%)
13529	 0.0	 0.0	(55.1%)
13530	 0.0	 0.0	(98.81%)
13531	 0.0	 0.0	(98.14%)
13532	 0.0	 0.0	(98.78%)
13533	 0.0	 0.0	(65.03%)
13534	 0.0	 0.0	(99.14%)
13535	 0.0	 0.0	(99.51%)
13536	 0.0	 0.0	(100.0%)
13537	 0.0	 0.0	(100.0%)
13538	 0.0	 0.0	(99.98%)
13539	 0.0	 0.0	(98.24%)
13540	 0.0	 0.0	(82.35%)
13541	 0.0	 0.0	(99.67%)
13542	 0.0	 0.0	(98.44%)
13543	 0.0	 0.0	(97.9%)
13544	 0.0	 1.0	(1.24%)
13545	 0.0	 0.0	(98.38%)
1354

13835	 0.0	 0.0	(99.74%)
13836	 0.0	 0.0	(82.04%)
13837	 0.0	 0.0	(99.84%)
13838	 0.0	 0.0	(100.0%)
13839	 0.0	 0.0	(98.87%)
13840	 0.0	 0.0	(92.93%)
13841	 0.0	 0.0	(99.97%)
13842	 0.0	 0.0	(99.98%)
13843	 0.0	 0.0	(99.91%)
13844	 0.0	 0.0	(99.9%)
13845	 0.0	 0.0	(99.8%)
13846	 0.0	 0.0	(94.8%)
13847	 0.0	 0.0	(98.55%)
13848	 0.0	 0.0	(69.99%)
13849	 0.0	 0.0	(98.97%)
13850	 0.0	 0.0	(84.96%)
13851	 0.0	 0.0	(99.69%)
13852	 0.0	 0.0	(99.7%)
13853	 0.0	 0.0	(99.95%)
13854	 0.0	 0.0	(99.93%)
13855	 0.0	 0.0	(99.67%)
13856	 0.0	 0.0	(99.66%)
13857	 0.0	 0.0	(99.87%)
13858	 0.0	 0.0	(99.99%)
13859	 0.0	 0.0	(99.86%)
13860	 0.0	 0.0	(95.22%)
13861	 0.0	 0.0	(95.76%)
13862	 0.0	 0.0	(80.51%)
13863	 0.0	 0.0	(100.0%)
13864	 0.0	 0.0	(99.99%)
13865	 0.0	 0.0	(99.99%)
13866	 0.0	 0.0	(100.0%)
13867	 0.0	 0.0	(99.87%)
13868	 0.0	 0.0	(99.76%)
13869	 0.0	 0.0	(98.92%)
13870	 0.0	 0.0	(99.83%)
13871	 0.0	 0.0	(99.93%)
13872	 0.0	 0.0	(99.82%)
13873	 0.0	 0.0	(99.93%)
13874	 0.0	 0.0	(99.98%)
1387

14164	 0.0	 0.0	(99.46%)
14165	 0.0	 0.0	(98.9%)
14166	 0.0	 0.0	(89.74%)
14167	 0.0	 0.0	(99.4%)
14168	 0.0	 0.0	(97.8%)
14169	 0.0	 0.0	(99.54%)
14170	 0.0	 0.0	(4.392%)
14171	 0.0	 0.0	(96.72%)
14172	 0.0	 0.0	(93.56%)
14173	 0.0	 0.0	(99.97%)
14174	 0.0	 0.0	(99.29%)
14175	 0.0	 0.0	(99.22%)
14176	 0.0	 0.0	(93.5%)
14177	 0.0	 0.0	(99.99%)
14178	 0.0	 0.0	(93.95%)
14179	 0.0	 0.0	(99.95%)
14180	 0.0	 0.0	(99.27%)
14181	 0.0	 0.0	(99.58%)
14182	 0.0	 0.0	(99.33%)
14183	 0.0	 0.0	(97.19%)
14184	 0.0	 0.0	(99.97%)
14185	 0.0	 0.0	(93.98%)
14186	 0.0	 0.0	(99.61%)
14187	 0.0	 0.0	(99.16%)
14188	 0.0	 0.0	(99.98%)
14189	 0.0	 0.0	(99.91%)
14190	 0.0	 0.0	(98.97%)
14191	 0.0	 0.0	(99.98%)
14192	 0.0	 0.0	(99.94%)
14193	 0.0	 0.0	(99.86%)
14194	 0.0	 0.0	(99.8%)
14195	 0.0	 0.0	(100.0%)
14196	 0.0	 0.0	(99.87%)
14197	 0.0	 0.0	(98.02%)
14198	 0.0	 0.0	(99.96%)
14199	 0.0	 0.0	(99.08%)
14200	 0.0	 0.0	(99.87%)
14201	 0.0	 0.0	(99.98%)
14202	 0.0	 0.0	(99.89%)
14203	 0.0	 0.0	(99.97%)
14204

In [72]:
print (float(wrongDetect) / _NUM_RECORDS )*100

87.6527777778
